# Battle of the Cities

---------------------------------------------------

This notebook contains the coding needed to find a suitable city to open a bookstore.

### The Buisness Dilemma:
An enthusiastic entrepreneur wants to open up a bookstore in populated city in Finland. Bookstores are vanishing these days due to online books or as they say ebooks but some people still prefer the reading from a book , this entrepreneur wants to bring back the feel of reading from a book . This project aims at finding an appropriate city to open a bookstore.This project can be used by anyone who wishes to open a bookstore or any other retail business in Finland.

### Data
The data needed for this project would be the name of the cities in Finland and their respective latitudes and longitudes, also the name, id , latitude and longitude of the respective venues in and around the cities of Finland. Data for the city like the names, latitude and longitude of the city will be taken from the website: Simple maps,Finland Cities Database, where the data is available in the form of a csv file and also the name, id , latitude and longitude of the respective venues in and around the cities is provided by the Foursquare API.

#### Packages that need to be installed for working the data
Geopy

Geocoder

Folium


#### Libraries needed for this project

numpy

pandas

random

requests

matplotlib

KMeans

folium

json_normalize

Nominatim

Installing nessesary packages

In [2]:
!pip install geopy
!pip install geocoder
!pip install folium

In [3]:
import pandas as pd
import numpy as np
import random # Library for random number generation
import requests # Library to handle requests
from pandas.io.json import json_normalize # transform JSON filee into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means for clustering state
from sklearn.cluster import KMeans

import folium # map rendering library

from geopy.geocoders import Nominatim # for getting latitude and longitude

print('Libraries imported.')

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

Libraries imported.


#### Main coding

Download the data

In [4]:
# data downloaded from website into a fi.csv file
!wget -q -O 'fi.csv' https://simplemaps.com/static/data/country-cities/fi/fi.csv
print('Data Downloaded')

Data Downloaded


Converting csv file into pandas dataframe

In [5]:
fincity_df= pd.read_csv('fi.csv')
print('Size of the dataframe downloaded is:',fincity_df.shape)
fincity_df

Size of the dataframe downloaded is: (323, 9)


,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Helsinki,60.1756,24.9342,Finland,FI,Uusimaa,primary,642045.0,642045.0
1,Espoo,60.2100,24.6600,Finland,FI,Uusimaa,minor,269802.0,269802.0
2,Tampere,61.4981,23.7608,Finland,FI,Pirkanmaa,admin,225118.0,225118.0
3,Vantaa,60.3000,25.0333,Finland,FI,Uusimaa,minor,214605.0,214605.0
4,Oulu,65.0142,25.4719,Finland,FI,Pohjois-Pohjanmaa,admin,200526.0,200526.0
...,...,...,...,...,...,...,...,...,...
318,Åva,60.4500,21.0833,Finland,FI,Åland,minor,NaN,NaN
319,Maaninka,63.1500,27.3000,Finland,FI,Pohjois-Savo,minor,NaN,NaN
320,Tammela,60.8000,23.7667,Finland,FI,Kanta-Häme,minor,NaN,NaN
321,Tohmajärvi,62.1833,30.3833,Finland,FI,Pohjois-Karjala,minor,NaN,NaN


Data pre-processing

Data pre-processing is done to get the desired data from the dataframe in the desired format for futher use of the data from the dataframe

Rows in the dataframe that have NaN values in the population column will be dropped as a good place would be a populated one.

In [6]:
# removing rows that have as population as NaN
fincity_df.dropna(subset=['population'], axis=0, inplace=True)
fincity_df.tail(5) # checking dataframe after drop

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
106,Salla,66.8333,28.6667,Finland,FI,Lappi,minor,3727.0,3727.0
107,Pello,66.7750,23.9667,Finland,FI,Lappi,minor,3623.0,3623.0
108,Kaavi,62.9750,28.4833,Finland,FI,Pohjois-Savo,minor,3194.0,3194.0
109,Muonio,67.9500,23.6833,Finland,FI,Lappi,minor,2358.0,2358.0
110,Kaskinen,62.3847,21.2222,Finland,FI,Pohjanmaa,minor,1285.0,1285.0


Some of the columns would not be needed like the columns: Country, iso2, population_proper. Hence these columns would be dropped

In [7]:
fincity_df = fincity_df.drop(columns =['country', 'iso2','population_proper'])
fincity_df.head(5)

,city,lat,lng,admin_name,capital,population
0,Helsinki,60.1756,24.9342,Uusimaa,primary,642045.0
1,Espoo,60.2100,24.6600,Uusimaa,minor,269802.0
2,Tampere,61.4981,23.7608,Pirkanmaa,admin,225118.0
3,Vantaa,60.3000,25.0333,Uusimaa,minor,214605.0
4,Oulu,65.0142,25.4719,Pohjois-Pohjanmaa,admin,200526.0


In [8]:
fincity_df.shape

(111, 6)

In [9]:
# renaming column names for better understanding
fincity_df.rename(columns={'lat' : 'Latitude', 'lng' : 'Longitude', 'admin_name' : 'Region(Finnish)'}, inplace=True) # renaming column names
fincity_df.head()

,city,Latitude,Longitude,Region(Finnish),capital,population
0,Helsinki,60.1756,24.9342,Uusimaa,primary,642045.0
1,Espoo,60.2100,24.6600,Uusimaa,minor,269802.0
2,Tampere,61.4981,23.7608,Pirkanmaa,admin,225118.0
3,Vantaa,60.3000,25.0333,Uusimaa,minor,214605.0
4,Oulu,65.0142,25.4719,Pohjois-Pohjanmaa,admin,200526.0


In [10]:
#removing cities having population less than 10000
lenth = fincity_df.shape[0]
for i in range(0, lenth):
    if (fincity_df['population'][i]<10000):
        fincity_df.drop([i], axis=0, inplace=True)

fincity_df.shape

(75, 6)

In [11]:
# Showing the capital of Finland and the capital of its respective region
tempdf = fincity_df
tempdf = tempdf.groupby('capital')
tempdf.get_group('primary')

,city,Latitude,Longitude,Region(Finnish),capital,population
0,Helsinki,60.1756,24.9342,Uusimaa,primary,642045.0


In [12]:
tempdf.get_group('admin')

,city,Latitude,Longitude,Region(Finnish),capital,population
2,Tampere,61.4981,23.7608,Pirkanmaa,admin,225118.0
4,Oulu,65.0142,25.4719,Pohjois-Pohjanmaa,admin,200526.0
5,Turku,60.4517,22.2700,Varsinais-Suomi,admin,187604.0
6,Jyväskylä,62.2333,25.7333,Keski-Suomi,admin,137368.0
7,Lahti,60.9833,25.6556,Päijät-Häme,admin,118119.0
8,Kuopio,62.8925,27.6783,Pohjois-Savo,admin,112117.0
9,Kouvola,60.8681,26.7042,Kymenlaakso,admin,85855.0
10,Pori,61.4847,21.7972,Satakunta,admin,85363.0
11,Joensuu,62.6000,29.7639,Pohjois-Karjala,admin,75514.0
12,Lappeenranta,61.0583,28.1861,Etelä-Karjala,admin,72875.0


The main dataframe is now ready

Proting the cities on the map of Finland using folium

In [13]:
# Getting Geographic Location of Finland that is Latitude and Longitude
address = 'Finland'

geolocator = Nominatim(user_agent='fin_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordenates of Finland are {}, {}.'.format(latitude, longitude))

The geographical coordenates of Finland are 63.2467777, 25.9209164.


In [14]:

# create map of Finland using latitude and longitude values
finmap = folium.Map(location=[latitude, longitude], zoom_start=6)

# add markers to map
for lat, lng, city in zip(fincity_df['Latitude'], fincity_df['Longitude'], fincity_df['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color='orange',
        fill_opacity=0.7,
        parse_html=False).add_to(finmap)  
    
finmap

The above map shows cities in Finland

Now moving on exploring the cities further

For exploring and finding the different cafes, book shops and other venues in the above cities, Foursquare API will be used.
Foursquare API requires user credentials for the use of the API.
The below cell contains credentials but are hidden from sharing and also replaced.

In [15]:
# @hidden_cell
CLIENT_ID = 'IERRNM3FGRP0C2CC2AQ1PE2P1A1BARN0XFCNKYTHAUTCZQCB' # your Foursquare ID
CLIENT_SECRET = 'PSPCYG01Y4FJPC2KV3IDYORXXRFICWVQYIIDCJL0HKUDUMZW' # your Foursquare Secret
VERSION = '20210401' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
print('Credentials Accepted')


Credentials Accepted


The following fuction is created to get the venues in and around all the cities.

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    fin_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    fin_venues.columns = ['city', 
                  'Lat', 
                  'Long', 
                  'Venue',
                  'Venue Id',       
                  'Venue Lat', 
                  'Venue Long', 
                  'Venue Category']
    
    return(fin_venues)

In [22]:
fin_venues = getNearbyVenues(names=fincity_df['city'],
                                   latitudes=fincity_df['Latitude'],
                                   longitudes=fincity_df['Longitude']
                                  )

Helsinki
Espoo
Tampere
Vantaa
Oulu
Turku
Jyväskylä
Lahti
Kuopio
Kouvola
Pori
Joensuu
Lappeenranta
Hämeenlinna
Vaasa
Rovaniemi
Seinäjoki
Mikkeli
Kotka
Salo
Porvoo
Kokkola
Lohja
Hyvinkää
Nurmijärvi
Järvenpää
Rauma
Kajaani
Savonlinna
Kerava
Nokia
Ylöjärvi
Kaarina
Riihimäki
Imatra
Sastamala
Raahe
Raisio
Iisalmi
Tornio
Kemi
Kurikka
Jämsä
Valkeakoski
Varkaus
Hamina
Äänekoski
Heinola
Jakobstad
Naantali
Pieksämäki
Forssa
Toijala
Kauhava
Loimaa
Orimattila
Kuusamo
Uusikaupunki
Pargas
Lovisa
Ylivieska
Lapua
Kauhajoki
Ulvila
Kalajoki
Alavus
Lieksa
Kankaanpää
Mariehamn
Nivala
Kitee
Paimio
Huittinen
Keuruu
Alajärvi


In [23]:
# Displaying the new dataframe and its size
print('Size of the new venues dataframe ',fin_venues.shape)
fin_venues.head(5)

Size of the new venues dataframe  (3387, 8)


,city,Lat,Long,Venue,Venue Id,Venue Lat,Venue Long,Venue Category
0,Helsinki,60.1756,24.9342,Sinisen huvilan kahvila,4be302eb63609c74cfd51bff,60.181305,24.937043,Café
1,Helsinki,60.1756,24.9342,Taidehalli,4adcdb23f964a520dc6021e3,60.172127,24.931014,Art Gallery
2,Helsinki,60.1756,24.9342,Cafe Rouge,5555afa9498efb7ce749253c,60.168711,24.933027,Middle Eastern Restaurant
3,Helsinki,60.1756,24.9342,Buongiorno Cafe & Restaurant,51289893e4b0386981d9e120,60.175304,24.919294,Café
4,Helsinki,60.1756,24.9342,Cafetoria,4f2634ede4b0020c308d539d,60.173203,24.923024,Coffee Shop


In [25]:
# Checking the number of venues returned for each city.
fin_venues.groupby('city').count()

,Lat,Long,Venue,Venue Id,Venue Lat,Venue Long,Venue Category
city,,,,,,,
Alajärvi,6,6,6,6,6,6,6
Alavus,11,11,11,11,11,11,11
Espoo,100,100,100,100,100,100,100
Forssa,24,24,24,24,24,24,24
Hamina,26,26,26,26,26,26,26
...,...,...,...,...,...,...,...
Vantaa,100,100,100,100,100,100,100
Varkaus,18,18,18,18,18,18,18
Ylivieska,16,16,16,16,16,16,16


In [27]:
# Finding out how many unique categories can be curated from all the returned venues
print('There are {} unique categories'.format(len(fin_venues['Venue Category'].unique())))

There are 296 unique categories


Analyzing Each City's Venues

In [28]:
# Using One-hot coding approach
# one hot encoding
fin_onehot = pd.get_dummies(fin_venues[['Venue Category']], prefix="", prefix_sep="")

# add city column back to dataframe
fin_onehot['city'] = fin_venues['city'] 

# move city column to the first column
fixed_columns = [fin_onehot.columns[-1]] + list(fin_onehot.columns[:-1])
fin_onehot = fin_onehot[fixed_columns]

fin_onehot.head(5)

,city,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Apres Ski Bar,Aquarium,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Helsinki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Helsinki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Helsinki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Helsinki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Helsinki,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# grouping rows by city and by taking the mean of the frequency of occurrence of each category
fin_group= fin_onehot.groupby('city').mean().reset_index()
fin_group.head(5)

,city,ATM,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Apres Ski Bar,Aquarium,...,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Wine Shop,Wings Joint,Yoga Studio,Zoo
0,Alajärvi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
1,Alavus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
2,Espoo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.01,0.0
3,Forssa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0
4,Hamina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0


In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Creating a dataframe with top 10 venues for each city.

In [31]:
num_top_venues = 10 # top 10 venues 

indicators = ['st', 'nd', 'rd']

# creating  columns according to number of top venues
columns = ['city']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# creating a new dataframe having city names and its top 10 venues
f_venues = pd.DataFrame(columns=columns)
f_venues['city'] = fin_group['city']

for ind in np.arange(fin_group.shape[0]):
    f_venues.iloc[ind, 1:] = return_most_common_venues(fin_group.iloc[ind, :], num_top_venues)

f_venues.head()  # top 10 venues dataframe

,city,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alajärvi,Supermarket,Discount Store,Home Service,Neighborhood,Other Great Outdoors,Organic Grocery,Opera House,North Indian Restaurant,Noodle House,Nightclub
1,Alavus,RV Park,Supermarket,Grocery Store,Clothing Store,Sandwich Place,Train Station,Café,Bus Station,Hotel,Department Store
2,Espoo,Café,Golf Course,Beach,Gym,Park,Gym / Fitness Center,Pizza Place,Coffee Shop,Pool,Supermarket
3,Forssa,Supermarket,Pizza Place,Fast Food Restaurant,Recreation Center,Bakery,Plaza,Cosmetics Shop,Restaurant,Farm,Nightclub
4,Hamina,Grocery Store,Supermarket,Hotel,Fish Market,Pub,Fast Food Restaurant,Café,Campground,Boat or Ferry,Ski Area


In [32]:
# checking new top 10 venues dataframe size
f_venues.shape

(75, 11)

### Use of KMeans Clustering

Usin KMeans to form clusters and find suitable cities, here 4 clusters are used

In [37]:
# set number of clusters
k = 4

# droping city column to get only top 10 venues columns
fin_clust = fin_group.drop('city', 1) 

# run k-means clustering
# fit kmean model with fin_clust dataframe
kmean = KMeans(n_clusters=k, random_state=0).fit(fin_clust)

# check cluster labels generated for each row in the dataframe
kmean.labels_[0:10]

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:56: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype = np.float
/srv/conda/envs/notebook/lib/python3.7/site-

array([1, 0, 0, 3, 3, 0, 0, 3, 0, 0], dtype=int32)

In [38]:
# add clustering labels
f_venues.insert(0, 'Cluster Labels', kmean.labels_)

# copying original city dataframe to new dataframe
findf = fincity_df

# merge city dataframe with city venues to add latitude/longitude for each city
findf = findf.join(f_venues.set_index('city'), on='city')

findf.head() # check new datframe and new cluster label column

,city,Latitude,Longitude,Region(Finnish),capital,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Helsinki,60.1756,24.9342,Uusimaa,primary,642045.0,0,Café,Park,Scandinavian Restaurant,Coffee Shop,French Restaurant,Hotel,Bakery,Farmers Market,Pizza Place,Sushi Restaurant
1,Espoo,60.2100,24.6600,Uusimaa,minor,269802.0,0,Café,Golf Course,Beach,Gym,Park,Gym / Fitness Center,Pizza Place,Coffee Shop,Pool,Supermarket
2,Tampere,61.4981,23.7608,Pirkanmaa,admin,225118.0,0,Café,Park,Restaurant,Gym / Fitness Center,Gastropub,Scenic Lookout,Scandinavian Restaurant,Sauna / Steam Room,Music Venue,Hotel
3,Vantaa,60.3000,25.0333,Uusimaa,minor,214605.0,0,Pizza Place,Recreation Center,Café,Gym / Fitness Center,Hotel,Gym,Nature Preserve,Airport Lounge,Pet Store,Beach
4,Oulu,65.0142,25.4719,Pohjois-Pohjanmaa,admin,200526.0,0,Café,Supermarket,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Restaurant,Indian Restaurant,Gym,Chinese Restaurant,Scandinavian Restaurant


Finally creating a cluster map

Plotting cities of Finland on the map of Finland in clusters

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=6)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(findf['Latitude'], findf['Longitude'], findf['city'], findf['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Results of Clusters

Cluster 1 :Leisure and Shopping

In [40]:


findf.loc[findf['Cluster Labels'] == 0,findf.columns[[0] + list(range(5, findf.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Helsinki,642045.0,0,Café,Park,Scandinavian Restaurant,Coffee Shop,French Restaurant,Hotel,Bakery,Farmers Market,Pizza Place,Sushi Restaurant
1,Espoo,269802.0,0,Café,Golf Course,Beach,Gym,Park,Gym / Fitness Center,Pizza Place,Coffee Shop,Pool,Supermarket
2,Tampere,225118.0,0,Café,Park,Restaurant,Gym / Fitness Center,Gastropub,Scenic Lookout,Scandinavian Restaurant,Sauna / Steam Room,Music Venue,Hotel
3,Vantaa,214605.0,0,Pizza Place,Recreation Center,Café,Gym / Fitness Center,Hotel,Gym,Nature Preserve,Airport Lounge,Pet Store,Beach
4,Oulu,200526.0,0,Café,Supermarket,Pizza Place,Fast Food Restaurant,Gym / Fitness Center,Restaurant,Indian Restaurant,Gym,Chinese Restaurant,Scandinavian Restaurant
5,Turku,187604.0,0,Gym / Fitness Center,Pizza Place,Café,Park,Bar,Gym,Scandinavian Restaurant,Chinese Restaurant,Pool,Restaurant
6,Jyväskylä,137368.0,0,Supermarket,Café,Scandinavian Restaurant,Park,Gym,Grocery Store,Coffee Shop,Hotel,Gym / Fitness Center,Playground
7,Lahti,118119.0,0,Café,Supermarket,Bar,Restaurant,Ski Area,Burger Joint,Pizza Place,Gym,Beach,Hotel
8,Kuopio,112117.0,0,Supermarket,Grocery Store,Fast Food Restaurant,Scandinavian Restaurant,Café,Pub,Gym / Fitness Center,Pizza Place,Italian Restaurant,Bar
10,Pori,85363.0,0,Fast Food Restaurant,Café,Gym / Fitness Center,Supermarket,Pizza Place,Grocery Store,Shopping Mall,Park,Bar,Italian Restaurant


Cluster 2 : Supermarkets and Transportational services

In [41]:
findf.loc[findf['Cluster Labels'] == 1,findf.columns[[0] + list(range(5, findf.shape[1]))]]

,city,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,Kauhava,16784.0,1,Supermarket,Fraternity House,Pizza Place,Discount Store,Neighborhood,Organic Grocery,Opera House,North Indian Restaurant,Noodle House,Nightclub
74,Alajärvi,10006.0,1,Supermarket,Discount Store,Home Service,Neighborhood,Other Great Outdoors,Organic Grocery,Opera House,North Indian Restaurant,Noodle House,Nightclub


Cluster 3 : Cafe's, Bars, Pubs and Resturants



In [42]:
findf.loc[findf['Cluster Labels'] == 2,findf.columns[[0] + list(range(5, findf.shape[1]))]]


,city,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Lieksa,11772.0,2,Supermarket,Bakery,Hotel,Train Station,ATM,New American Restaurant,Organic Grocery,Opera House,North Indian Restaurant,Noodle House
67,Kankaanpää,11769.0,2,Supermarket,Chinese Restaurant,Campground,Kebab Restaurant,Pizza Place,Hot Spring,National Park,Nature Preserve,Neighborhood,New American Restaurant
69,Nivala,10876.0,2,Supermarket,Hotel,Burger Joint,Discount Store,Hockey Rink,Pizza Place,Train Station,ATM,Neighborhood,Opera House


Cluster 4 : Markets

In [44]:
findf.loc[findf['Cluster Labels'] == 3,findf.columns[[0] + list(range(5, findf.shape[1]))]]


,city,population,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Kouvola,85855.0,3,Supermarket,Fast Food Restaurant,Pizza Place,Café,Coffee Shop,Sandwich Place,Grocery Store,Pub,Chinese Restaurant,Beach
26,Rauma,39809.0,3,Supermarket,Café,Grocery Store,Fast Food Restaurant,Pizza Place,Restaurant,Turkish Restaurant,Hockey Arena,Steakhouse,Chinese Restaurant
28,Savonlinna,35523.0,3,Supermarket,Hotel,Pizza Place,Brewery,Train Station,Seafood Restaurant,Scandinavian Restaurant,Ski Area,Fast Food Restaurant,Resort
35,Sastamala,25220.0,3,Fast Food Restaurant,Supermarket,Café,Theme Park,Sandwich Place,Hotel,Gas Station,Pizza Place,Bakery,Grocery Store
36,Raahe,25165.0,3,Supermarket,Fast Food Restaurant,Flea Market,Grocery Store,Chinese Restaurant,Sandwich Place,Buffet,Café,Nightclub,Theater
38,Iisalmi,21945.0,3,Supermarket,Stables,Campground,Grocery Store,Fast Food Restaurant,Bar,Park,Liquor Store,Train Station,Golf Course
39,Tornio,21928.0,3,Supermarket,Chinese Restaurant,Smoke Shop,Grocery Store,Liquor Store,Shopping Mall,Clothing Store,Candy Store,Sandwich Place,Coffee Shop
40,Kemi,21758.0,3,Supermarket,Grocery Store,Hotel,Gas Station,Beer Bar,Harbor / Marina,Fast Food Restaurant,Sandwich Place,Train Station,Boat or Ferry
41,Kurikka,21734.0,3,Supermarket,Hockey Arena,Grocery Store,Pizza Place,Discount Store,Gas Station,Gym,Restaurant,New American Restaurant,Organic Grocery
42,Jämsä,21542.0,3,Supermarket,Ski Area,Apres Ski Bar,Pizza Place,Restaurant,Café,Sandwich Place,Shoe Store,Shopping Mall,Train Station


Resulting suitable cities



In [45]:
#list of cities suitable for opening a bookstore
suit_fincity = findf.loc[findf['Cluster Labels'] == 0,findf.columns[[0] + list(range(5, findf.shape[1]))]]
suit_list = suit_fincity['city'].values.tolist()
suit_list

['Helsinki',
 'Espoo',
 'Tampere',
 'Vantaa',
 'Oulu',
 'Turku',
 'Jyväskylä',
 'Lahti',
 'Kuopio',
 'Pori',
 'Joensuu',
 'Lappeenranta',
 'Hämeenlinna',
 'Vaasa',
 'Rovaniemi',
 'Seinäjoki',
 'Mikkeli',
 'Kotka',
 'Salo',
 'Porvoo',
 'Kokkola',
 'Lohja',
 'Hyvinkää',
 'Nurmijärvi',
 'Järvenpää',
 'Kajaani',
 'Kerava',
 'Nokia',
 'Ylöjärvi',
 'Kaarina',
 'Riihimäki',
 'Imatra',
 'Raisio',
 'Valkeakoski',
 'Heinola',
 'Jakobstad',
 'Naantali',
 'Toijala',
 'Orimattila',
 'Pargas',
 'Ulvila',
 'Alavus',
 'Mariehamn',
 'Paimio']

## Conclusion


Results from the KMeans cluster shows 4 clusters:

- Cluster 1: Shows venues fit for family leisure like parks , theatres, bookstore, hotels, etc.
- Cluster 2: Shows sumermarkets and transport services
- Cluster 3 : Shows cafe's, bars, pubs and resturants and all high end leisure venues.
- Cluster 4 : Shows markets like sumpermarkets fish market

From the above analysis and resulting clusters, a conclusion can be drawn that cities in cluster 1 are more suitable to openup a bookstore as the cities in cluster 1 are fairly populated and is advisable for the group of people that the owner plans on targeting. Hence, Cities in Cluster 1 are Suitable for opening a bookstore. These cities are :
Kuopio, Kouvola, Joensuu, Seinäjoki, Mikkeli, Kokkola, Järvenpää, Rauma, Kajaani, Savonlinna, Kerava, Nokia, Riihimäki, Sastamala, Raahe, Iisalmi, Tornio, Kemi, Kurikka, Jämsä, Valkeakoski, Hamina, Äänekoski, Heinola, Jakobstad, Pieksämäki, Forssa, Toijala, Loimaa, Orimattila, Kuusamo, Uusikaupunki, Lovisa, Ylivieska, Lapua, Kauhajoki, Ulvila, Kalajoki, Paimio, Huittinen, Keuruu